In [1]:
%reload_ext autoreload
%autoreload 2

import argparse
import os
import sys
import numpy as np
from timeit import default_timer as timer
from _collections import OrderedDict
import shutil
from pprint import pprint
start = timer()
HOME = os.path.expanduser("~")
PATH = os.path.join(HOME, 'programming/pipeline_utility')
sys.path.append(PATH)
from utilities.sqlcontroller import SqlController
from utilities.file_location import FileLocationManager
from utilities.utilities_cvat_neuroglancer import get_structure_number, NumpyToNeuroglancer, get_segment_properties
from utilities.utilities_affine import rigid_transform_3D, ralign, affine_fit, superalign, umeyama


Connecting dklab@192.168.1.12:3306


In [2]:
animal = 'DK52'
fileLocationManager = FileLocationManager(animal)
atlas_name = 'atlasV7'
DATA_PATH = '/net/birdstore/Active_Atlas_Data/data_root'
ROOT_DIR = '/net/birdstore/Active_Atlas_Data/data_root/pipeline_data'
THUMBNAIL_DIR = os.path.join(ROOT_DIR, animal, 'preps', 'CH1', 'thumbnail')
ATLAS_PATH = os.path.join(DATA_PATH, 'atlas_data', atlas_name)
ORIGIN_PATH = os.path.join(ATLAS_PATH, 'origin')
VOLUME_PATH = os.path.join(ATLAS_PATH, 'structure')
OUTPUT_DIR = os.path.join(fileLocationManager.neuroglancer_data, 'atlas')
#if os.path.exists(OUTPUT_DIR):
#    shutil.rmtree(OUTPUT_DIR)
os.makedirs(OUTPUT_DIR, exist_ok=True)
origin_files = sorted(os.listdir(ORIGIN_PATH))
volume_files = sorted(os.listdir(VOLUME_PATH))
sqlController = SqlController(animal)
resolution = sqlController.scan_run.resolution
surface_threshold = 0.8
SCALE = (10 / resolution)

structure_volume_origin = {}
for volume_filename, origin_filename in zip(volume_files, origin_files):
    structure = os.path.splitext(volume_filename)[0]
    if structure not in origin_filename:
        print(structure, origin_filename)
        break

    color = get_structure_number(structure.replace('_L', '').replace('_R', ''))

    origin = np.loadtxt(os.path.join(ORIGIN_PATH, origin_filename))
    volume = np.load(os.path.join(VOLUME_PATH, volume_filename))

    volume = np.rot90(volume, axes=(0, 1))
    volume = np.flip(volume, axis=0)
    volume[volume > surface_threshold] = color
    volume = volume.astype(np.uint8)

    structure_volume_origin[structure] = (volume, origin)

col_length = sqlController.scan_run.width/SCALE
row_length = sqlController.scan_run.height/SCALE
z_length = len(os.listdir(THUMBNAIL_DIR))
atlasV7_volume = np.zeros(( int(row_length), int(col_length), z_length), dtype=np.uint8)
print('atlas volume shape', atlasV7_volume.shape)

DK52_centers = {'12N': [46488, 18778, 242],
                '5N_L': [38990, 20019, 172],
                '5N_R': [39184, 19027, 315],
                '7N_L': [42425, 23190, 166],
                '7N_R': [42286, 22901, 291]}
centers = OrderedDict(DK52_centers)
centers_list = []
for value in centers.values():
    centers_list.append((value[1]/SCALE, value[0]/SCALE, value[2]))
COM = np.array(centers_list)
atlas_com_centers = OrderedDict()
atlas_all_centers = {}
for structure, (volume, origin) in sorted(structure_volume_origin.items()):
    midcol, midrow, midz = origin
    row_start = midrow + row_length / 2
    col_start = midcol + col_length / 2
    z_start = midz / 2 + z_length / 2
    row_end = row_start + volume.shape[0]
    col_end = col_start + volume.shape[1]
    z_end = z_start + (volume.shape[2] + 1) / 2
    midcol = (col_end + col_start) / 2
    midrow = (row_end + row_start) / 2
    midz = (z_end + z_start) / 2
    if structure in centers.keys():
        atlas_com_centers[structure] = [midrow, midcol, midz]
    atlas_all_centers[structure] = [midrow, midcol, midz]
ATLAS_centers = OrderedDict(atlas_com_centers)
ATLAS = np.array(list(ATLAS_centers.values()))

atlas volume shape (1170, 2112, 486)


In [ ]:
# good visual results
R, t = rigid_transform_3D(ATLAS.T, COM.T)
if t.shape[0] == 3:
    t = np.reshape(t, (1,3))
pprint(R)
pprint(t)

In [ ]:
# ok visual, needed translation
R, t = ralign(ATLAS, COM)
if t.shape[0] == 3:
    t = np.reshape(t, (1,3))

In [ ]:
# good visual results
R, t = superalign(ATLAS, COM)
if t.shape[0] == 3:
    t = np.reshape(t, (1,3))

In [ ]:
c, R, t = umeyama(ATLAS, COM)
if t.shape[0] == 3:
    t = np.reshape(t, (1,3))

In [ ]:
# bad results
trn = affine_fit(ATLAS, COM)

In [3]:
# gives bad results, volumes overlap, too close
animal_centroid = np.mean(COM, axis=0)
atlas_centroid = np.mean(ATLAS, axis=0)
t = animal_centroid - atlas_centroid
X = np.linalg.inv(COM.T @ COM) @ COM.T @ ATLAS
U, S, V = np.linalg.svd(X)
R1 = np.dot(U, V)
R = np.array([[np.cos(R1[0][0]), -np.sin(R1[0][1]), 0],
              [np.sin(R1[1][0]), np.cos(R1[1][1]), 0],
              [0, 0, 1]])
cx = S[0]
cy = S[1]
s = np.array([[cx, 0, 0], [0, cy, 0], [0, 0, 1]])

In [ ]:
R = np.array([
    [0.89, 0.475, -0.024],
    [-0.3596, 1.173566, -0.0858],
    [-0.0083, 0.09963,1.12647]
])
t = np.array([18917.2/SCALE, 6900/SCALE, 48.674])
if t.shape[0] == 3:
    t = np.reshape(t, (1, 3))
s = 1

In [4]:
for structure, (volume, origin) in sorted(structure_volume_origin.items()):
    print(str(structure).ljust(6),end=": ")
    x,y,z = origin
    arr = np.array([y,x,z])
    input = arr + t
    results = np.dot(s*R, input.T).reshape(3,1)
    y = results[0][0]
    x = results[1][0]
    z = results[2][0]
    x_start = int( round(x + col_length / 2))
    y_start = int( round(y + row_length / 2))
    z_start = int( round(z / 2 + z_length / 2))
    x_end = x_start + volume.shape[0]
    y_end = y_start + volume.shape[1]
    z_end = int( z_start + (volume.shape[2] + 1) / 2)
    print('Row', str(y_start).rjust(4), str(y_end).rjust(4),'Col', str(x_start).rjust(4), str(x_end).rjust(4),
          'Z', str(z_start).rjust(4), str(z_end).rjust(4), end=" ")

        
    z_indices = [z for z in range(volume.shape[2]) if z % 2 == 0]
    volume = volume[:, :, z_indices]
    volume = np.swapaxes(volume, 0, 1)

    try:
        atlasV7_volume[y_start:y_end, x_start:x_end, z_start:z_end] += volume
    except ValueError as ve:
        print("Error", end=" ")

    print()
# rotation puts in view with wide x and shallow  y
atlasV7_volume = np.rot90(atlasV7_volume, 1)
# fliplr and flip axis=1 did not work, axis=0 works
atlasV7_volume = np.flip(atlasV7_volume, axis=0)
#atlasV7_volume = affine_transform(atlasV7_volume, R)
print('Done with big volume:', atlasV7_volume.shape)

10N_L : Row  608  696 Col 1286 1416 Z  212  239 
10N_R : Row  608  696 Col 1286 1416 Z  241  268 
12N   : Row  615  693 Col 1283 1407 Z  218  261 
3N_L  : Row  522  563 Col 1116 1179 Z  220  246 
3N_R  : Row  522  563 Col 1116 1179 Z  234  260 
4N_L  : Row  530  553 Col 1147 1166 Z  223  234 
4N_R  : Row  530  553 Col 1147 1166 Z  247  258 
5N_L  : Row  573  659 Col 1162 1235 Z  157  187 
5N_R  : Row  573  659 Col 1162 1235 Z  294  324 
6N_L  : Row  597  622 Col 1208 1234 Z  215  228 
6N_R  : Row  597  622 Col 1208 1234 Z  252  265 
7N_L  : Row  645  721 Col 1196 1297 Z  159  206 
7N_R  : Row  645  721 Col 1196 1297 Z  275  322 
7n_L  : Row  593  701 Col 1173 1280 Z  151  226 
7n_R  : Row  593  701 Col 1173 1280 Z  254  329 
AP    : Row  601  643 Col 1298 1365 Z  230  252 
Amb_L : Row  651  678 Col 1251 1319 Z  175  182 
Amb_R : Row  651  678 Col 1251 1319 Z  299  306 
DC_L  : Row  552  640 Col 1208 1351 Z  110  184 
DC_R  : Row  552  640 Col 1208 1351 Z  297  371 
IC    : Row  399  58

In [ ]:
atlasV7_volume.shape

In [ ]:
%%time
ng = NumpyToNeuroglancer(atlasV7_volume, [10000, 10000, 20000])
ng.preview()
ng.init_precomputed(OUTPUT_DIR)
ng.add_segment_properties(get_segment_properties())
ng.add_downsampled_volumes()
ng.add_segmentation_mesh()